In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts3vs0_time_course.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 92 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(92): SRR7817611 SRR7817612 ... SRR4317647 SRR4317648
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 92
                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001   15.60781   15.62665   15.57097   15.64101   15.61714
ENSMUSG00000000028   10.39609   10.26505   10.49217   10.35018   10.41462
ENSMUSG00000000049   19.69053   19.67113   19.92672   19.89439   19.89839
ENSMUSG00000000058   10.94494   11.07184   11.10158   10.84190   11.32334
ENSMUSG00000000085   12.49166   12.27902   12.41129   12.41011   12.10090
ENSMUSG00000000088   16.55014   16.55623   16.52634   16.25109   16.20891
                   SRR7817623 SRR7817624 SRR7817625 SRR7817626 SRR7817627
ENSMUSG00000000001   15.55034   15.57425   15.71061   15.69143   15.53124
ENSMUSG00000000028   10.56152   10.38823   10.29158   10.44353   10.92519
ENSMUSG00000000049   19.91441   19.88572   19.82340   19.83530   19.85309
ENSMUSG00000000058   11.08883   11.41464   10.88050   11.03490   10.99547
ENSMUSG00000000085   12.24541   12.47628   12.56159   12.42722   12.15383
ENSMUSG00000000088   16.16684  

ENSMUSG00000000085   12.22338   12.46157   12.14364   12.03328   12.42148
ENSMUSG00000000088   15.76521   15.74539   15.10864   14.96532   14.71069
                   SRR4317608 SRR4317645 SRR4317607 SRR4317609 SRR4317610
ENSMUSG00000000001   14.93861   15.33209   15.03243   15.25888   15.35722
ENSMUSG00000000028   10.95089   10.87317   10.54268   10.79650   10.62579
ENSMUSG00000000049   20.04603   19.88684   20.04833   20.11295   20.12268
ENSMUSG00000000058   11.13118   11.64691   11.47908   11.60663   11.81357
ENSMUSG00000000085   12.44834   12.50120   12.54457   11.93590   12.25633
ENSMUSG00000000088   15.10656   15.02666   15.11629   14.84659   15.00401
                   SRR4317611 SRR4317612 SRR4317643 SRR4317644 SRR4317646
ENSMUSG00000000001   15.19602   15.09000   15.36009   15.32580   15.30897
ENSMUSG00000000028   10.87312   10.55382   10.83767   10.80558   10.68661
ENSMUSG00000000049   20.09947   20.09969   19.89836   19.89709   19.88099
ENSMUSG00000000058   11.42457   11.695

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec3vs0_time_course.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[39] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 0 0 0 3 3 3 3 3 0 0 0 0 3 0 3 3
[77] 3 3 3 3 0 3 0 0 0 0 0 3 3 3 3 3


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls3vs0_time_course.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[31] "ENSMUSG00000093838" "ENSMUSG00000093861" "ENSMUSG00000094029"
[34] "ENSMUSG00000094134" "ENSMUSG00000094174" "ENSMUSG00000094194"
[37] "ENSMUSG00000094335" "ENSMUSG00000094491" "ENSMUSG00000094505"
[40] "ENSMUSG00000094951" "ENSMUSG00000095079" "ENSMUSG00000095117"
[43] "ENSMUSG00000095170" "ENSMUSG00000095210" "ENSMUSG00000095285"
[46] "ENSMUSG00000095429" "ENSMUSG00000095497" "ENSMUSG00000095571"
[49] "ENSMUSG00000095612" "ENSMUSG00000095737" "ENSMUSG00000096336"
[52] "ENSMUSG00000096407" "ENSMUSG00000096580" "ENSMUSG00000102301"
[55] "ENSMUSG00000102364" "ENSMUSG00000103033" "ENSMUSG00000103939"
[58] "ENSMUSG00000104452" "ENSMUSG00000105606" "ENSMUSG00000105906"
[61] "ENSMUSG00000106403"

$`R-MMU-448708`
[1] "ENSMUSG00000026420" "ENSMUSG00000037157" "ENSMUSG00000044244"

$`R-MMU-8987012`
[1] "ENSMUSG00000026420" "ENSMUSG00000037157" "ENSMUSG00000044244"

$`R-MMU-5660470`
[1] "ENSMUSG00000026421" "ENSMUSG00000028890"

$`R-MMU-5660533`
[1] "ENSMUSG00000026421" "ENSMUSG0000002889

[1] "ENSMUSG00000026778" "ENSMUSG00000027164" "ENSMUSG00000028191"
[4] "ENSMUSG00000032688" "ENSMUSG00000036526"

$`R-MMU-2730899`
[1] "ENSMUSG00000026778" "ENSMUSG00000028191" "ENSMUSG00000032688"
[4] "ENSMUSG00000036526"

$`R-MMU-2730902`
[1] "ENSMUSG00000026778" "ENSMUSG00000036526"

$`R-MMU-2730903`
[1] "ENSMUSG00000026778" "ENSMUSG00000027164" "ENSMUSG00000028191"
[4] "ENSMUSG00000032688" "ENSMUSG00000036526"

$`R-MMU-374664`
[1] "ENSMUSG00000026778" "ENSMUSG00000052397"

$`R-MMU-9653503`
[1] "ENSMUSG00000026778" "ENSMUSG00000030265"

$`R-MMU-8848247`
[1] "ENSMUSG00000026781" "ENSMUSG00000056938"

$`R-MMU-2162253`
[1] "ENSMUSG00000026784" "ENSMUSG00000038240"

$`R-MMU-354060`
[1] "ENSMUSG00000026786" "ENSMUSG00000052681" "ENSMUSG00000068798"

$`R-MMU-354097`
[1] "ENSMUSG00000026786" "ENSMUSG00000028465" "ENSMUSG00000052681"
[4] "ENSMUSG00000068798"

$`R-MMU-888577`
[1] "ENSMUSG00000026787" "ENSMUSG00000070880"

$`R-MMU-9022928`
[1] "ENSMUSG00000026787" "ENSMUSG00000031393"

$`R-MM

[7] "ENSMUSG00000054204" "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-201521`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000054204"
[4] "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-9667914`
[1] "ENSMUSG00000027239" "ENSMUSG00000055471"

$`R-MMU-9700120`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000068748"

$`R-MMU-9700121`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000068748"

$`R-MMU-9700154`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000042626"
[4] "ENSMUSG00000054204" "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-9700156`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000054204"
[4] "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-9700168`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000054204"
[4] "ENSMUSG00000055471" "ENSMUSG00000087247"

$`R-MMU-9700175`
[1] "ENSMUSG00000027239" "ENSMUSG00000029838" "ENSMUSG00000042626"
[4] "ENSMUSG00000054204" "ENSMUSG00000055471" "ENSMUSG00000087247"

$`

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000076731"
[28] "ENSMUSG00000076733" "ENSMUSG00000076937" "ENSMUSG00000087582"
[31] "ENSMUSG00000093838" "ENSMUSG00000093861" "ENSMUSG00000094029"
[34] "ENSMUSG00000094134" "ENSMUSG00000094174" "ENSMUSG00000094194"
[37] "ENSMUSG00000094335" "ENSMUSG00000094491" "ENSMUSG00000094505"
[40] "ENSMUSG00000094951" "ENSMUSG00000095079" "ENSMUSG00000095117"
[43] "ENSMUSG00000095170" "ENSMUSG00000095210" "ENSMUSG00000095285"
[46] "ENSMUSG00000095429" "ENSMUSG00000095497" "ENSMUSG00000095571"
[49] "ENSMUSG00000095612" "ENSMUSG00000095737" "ENSMUSG00000096336"
[52] "ENSMUSG00000096407" "ENSMUSG00000096580" "ENSMUSG00000102301"
[55] "ENSMUSG00000102364" "ENSMUSG00000103033" "ENSMUSG00000103939"
[58] "ENSMUSG00000104452" "ENSMUSG00000105606" "ENSMUSG00000105906"
[61] "ENSMUSG00000106403"

$`R-MMU-8856189`
[1] "ENSMUSG00000028362" "ENSMUSG00000028602"

$`R-MMU-1250256`
[1] "ENSMUSG00000028367" "ENSMUSG00000038393"

$`R-MMU-1250264`
[1] "ENSMUSG00000028367" "ENSMUSG00000038393"

$`R-MMU-125

In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

  15.641011  15.617142
ENSMUSG00000000028  10.396094  10.265050  10.492169  10.350183  10.414622
ENSMUSG00000000049  19.690525  19.671133  19.926724  19.894393  19.898387
ENSMUSG00000000058  10.944936  11.071840  11.101575  10.841896  11.323345
ENSMUSG00000000085  12.491665  12.279023  12.411287  12.410110  12.100900
ENSMUSG00000000088  16.550143  16.556226  16.526341  16.251088  16.208913
ENSMUSG00000000120  11.913453  11.968411  12.012620  11.668609  11.614271
ENSMUSG00000000126  10.180428  10.499808  10.067771  10.545789  10.406624
ENSMUSG00000000127  11.864985  11.693808  11.950413  11.861412  12.085904
ENSMUSG00000000142  12.022831  12.005993  11.988703  12.652696  12.438059
ENSMUSG00000000149  15.840580  15.905314  15.875935  15.977595  15.783469
ENSMUSG00000000154  15.378835  15.459865  15.528023  15.575085  15.587304
ENSMUSG00000000168  14.747357  14.773399  15.121462  15.073903  15.063199
ENSMUSG00000000171  16.876654  16.882846  17.010355  16.940567  16.935643
ENSMUSG00000000

ENSMUSG00000001833  14.346804  14.304428  14.329488  14.485515  14.498228
ENSMUSG00000001847  15.693921  15.671442  15.796252  15.749419  15.733991
ENSMUSG00000001855  12.821004  12.745312  12.434779  12.690686  12.662952
ENSMUSG00000001865   9.297616   9.822465   9.853552   9.297195   9.297148
ENSMUSG00000001870  11.027000  10.838986  10.936221  10.889425  10.825326
ENSMUSG00000001891  16.675175  16.729545  17.315805  17.028957  17.035565
ENSMUSG00000001901   9.297616   9.297080   9.287907   9.380619   9.297148
ENSMUSG00000001918  11.670859  11.643971  11.334483  11.207186  11.163355
ENSMUSG00000001924  16.745781  16.770143  16.591458  16.684709  16.711078
ENSMUSG00000001930  11.870447  12.052516  11.382124  11.664149  11.625941
ENSMUSG00000001946  11.940597  11.949270  12.074190  11.991476  12.335660
ENSMUSG00000001962  13.752698  13.594749  13.854539  13.680279  13.459133
ENSMUSG00000001964  12.767630  12.818219  13.135543  12.944247  12.954562
ENSMUSG00000001983  13.362949  13.2471

ENSMUSG00000002992  16.039021  16.106648  16.397450  16.039491  16.020292
ENSMUSG00000003031  13.605892  13.393079  13.707703  13.704967  13.495262
ENSMUSG00000003032  10.630891  10.495279  10.067771  10.841248  10.405619
ENSMUSG00000003033  13.828635  13.855202  13.838942  13.894001  13.809293
ENSMUSG00000003037  15.182052  15.238458  15.120592  15.063373  15.047130
ENSMUSG00000003039  14.819802  14.588722  14.525931  14.627116  14.749709
ENSMUSG00000003051  10.590870  10.538734  10.394499  10.539728  10.866550
ENSMUSG00000003053  19.525276  19.533921  19.914589  20.075082  20.074243
ENSMUSG00000003062  12.493702  12.434864  12.324635  12.415820  12.372263
ENSMUSG00000003068  15.588852  15.639021  15.680070  15.604442  15.591939
ENSMUSG00000003070   9.297616   9.920460   9.994782   9.850516   9.297148
ENSMUSG00000003072  17.200890  17.166844  17.009926  17.090397  17.106607
ENSMUSG00000003099  14.520713  14.599460  14.533120  14.600528  14.525273
ENSMUSG00000003119  12.955164  12.9774

ENSMUSG00000004187  12.192344  12.451002  12.319428  12.430532  12.327435
ENSMUSG00000004207  17.984657  17.992626  17.802177  17.894229  17.869040
ENSMUSG00000004221  15.934732  15.843083  16.002136  15.824004  15.882165
ENSMUSG00000004231   9.297616   9.297080   9.287907   9.297195   9.297148
ENSMUSG00000004263  13.370991  13.395711  13.211899  13.114937  13.221999
ENSMUSG00000004264  16.381342  16.347387  16.485054  16.467213  16.464488
ENSMUSG00000004266  13.586567  13.704379  13.281084  13.664019  13.634664
ENSMUSG00000004267  10.031436  10.103631  10.067771   9.297195   9.935245
ENSMUSG00000004268  14.555914  14.447979  14.746209  14.703087  14.577492
ENSMUSG00000004270  15.634953  15.744331  15.929557  15.893900  15.843845
ENSMUSG00000004285  13.762191  13.578438  13.635879  13.744677  13.639480
ENSMUSG00000004296   9.826457   9.297080   9.616591   9.297195   9.297148
ENSMUSG00000004317  11.846634  11.918370  12.244318  12.010438  11.807628
ENSMUSG00000004319  14.727693  14.7352

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ENSMUSG00000000058  11.876353  11.824889
ENSMUSG00000000085  12.265762  12.064808
ENSMUSG00000000088  14.950251  14.978766
ENSMUSG00000000120  10.411260  10.465851
ENSMUSG00000000126  10.345846  10.275551
ENSMUSG00000000127  12.062730  11.896302
ENSMUSG00000000142  11.877854  11.743776
ENSMUSG00000000149  15.988548  15.850354
ENSMUSG00000000154  15.822124  15.736888
ENSMUSG00000000168  14.649759  14.650741
ENSMUSG00000000171  16.741819  16.673040
ENSMUSG00000000182   9.885503   9.716898
ENSMUSG00000000183   9.305632   9.305693
ENSMUSG00000000184  12.530539  12.418072
ENSMUSG00000000197   9.305632   9.305693
ENSMUSG00000000214   9.305632   9.305693
ENSMUSG00000000215   9.305632   9.305693
ENSMUSG00000000216   9.305632   9.305693
ENSMUSG00000000223   9.305632   9.305693
ENSMUSG00000000247   9.885503  10.683245
ENSMUSG00000000253  10.812090  10.195802
ENSMUSG00000000263   9.305632   9.305693
ENSMUSG00000000275  15.700460  15.595698
ENSMUSG00000000276  11.420699  11.180613
ENSMUSG000000002

ENSMUSG00000002043  12.762687  12.955158
ENSMUSG00000002052  14.623566  14.713986
ENSMUSG00000002059  10.610704  10.515496
ENSMUSG00000002068  10.812927  10.526670
ENSMUSG00000002100   9.305632   9.305693
ENSMUSG00000002102  16.038159  16.028595
ENSMUSG00000002108  15.343197  15.429082
ENSMUSG00000002109  13.165618  12.946371
ENSMUSG00000002111  11.119568  11.388871
ENSMUSG00000002129  13.618859  13.805650
ENSMUSG00000002147  14.759905  14.753503
ENSMUSG00000002204  10.107603  10.275551
ENSMUSG00000002205  13.570992  13.675307
ENSMUSG00000002210  12.108921  12.106807
ENSMUSG00000002221  13.086429  13.036510
ENSMUSG00000002227  13.167453  13.322908
ENSMUSG00000002233  11.631876  11.467561
ENSMUSG00000002249  12.066051  12.084308
ENSMUSG00000002250  11.850611  11.844213
ENSMUSG00000002257   9.819417  10.314689
ENSMUSG00000002279  14.344451  14.487822
ENSMUSG00000002289  16.369610  16.445702
ENSMUSG00000002297  10.963822  10.855078
ENSMUSG00000002307  12.594108  12.588917
ENSMUSG000000023

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  16.273099  16.248417
ENSMUSG00000009575  14.525590  14.571851
ENSMUSG00000009585  11.467608  11.690690
ENSMUSG00000009588   9.305632   9.305693
ENSMUSG00000009614  18.799816  18.789513
ENSMUSG00000009621  14.440672  14.450099
ENSMUSG00000009630  12.852701  12.997275
ENSMUSG00000009647  12.853916  12.712870
ENSMUSG00000009681  13.900209  13.816297
ENSMUSG00000009731   9.716557   9.305693
ENSMUSG00000009863  17.006582  17.059772
ENSMUSG00000009900   9.305632   9.305693
ENSMUSG00000009905  14.281762  14.260737
ENSMUSG00000009907  14.705977  14.631688
ENSMUSG00000009927  16.195892  16.197775
ENSMUSG00000009941   9.305632   9.716898
ENSMUSG00000009995  14.118242  13.946154
ENSMUSG00000010021   9.716557   9.305693
ENSMUSG00000010025  17.668238  17.644802
ENSMUSG00000010045  13.234860  13.290156
ENSMUSG00000010047  14.086177  14.124786
ENSMUSG00000010051  15.051792  15.088887
ENSMUSG00000010057  12.377706  12.218319
ENSMUSG00000010064  19.414155  19.393661
ENSMUSG00000010066   9.571527   9.

ENSMUSG00000015787   9.305632   9.305693
ENSMUSG00000015790  14.340534  14.179327
ENSMUSG00000015806  17.184708  17.051460
ENSMUSG00000015812  10.067143   9.923578
ENSMUSG00000015829   9.305632   9.305693
ENSMUSG00000015837  16.843092  16.713178
ENSMUSG00000015839  15.683493  15.616189
ENSMUSG00000015843  11.913434  11.723935
ENSMUSG00000015846  17.968519  18.023932
ENSMUSG00000015850  11.527818  11.419546
ENSMUSG00000015880  10.662197  10.390362
ENSMUSG00000015889  13.738273  13.796273
ENSMUSG00000015890  15.820226  15.776852
ENSMUSG00000015937  14.441921  14.447610
ENSMUSG00000015944  11.194001  11.286831
ENSMUSG00000015947  11.006981  11.317650
ENSMUSG00000015950  11.533511  11.494626
ENSMUSG00000015957   9.305632   9.305693
ENSMUSG00000015961  14.052009  14.083454
ENSMUSG00000015968  14.450708  14.508244
ENSMUSG00000015970  17.330686  17.329314
ENSMUSG00000015971  14.076425  14.017597
ENSMUSG00000015994  13.406954  13.119120
ENSMUSG00000016018  13.730592  13.570109
ENSMUSG000000160

In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12
[13,]        13
[14,]        14
[15,]        15
[16,]        16
[17,]        17
[18,]        18
[19,]        19
[20,]        20
[21,]        21
[22,]        22
[23,]        23
[24,]        24
[25,]        25
[26,]        26
[27,]        27
[28,]        28
[29,]        29
[30,]        30
[31,]        31
[32,]        32
[33,]        33
[34,]        34
[35,]        35
[36,]        36
[37,]        37
[38,]        38
[39,]        39
[40,]        40
[41,]        41
[42,]        42
[43,]        43
[44,]        44
[45,]        45
[46,]        46
[47,]        47
[48,]        48
[49,]        49
[50,]        50
[51,]        51
[52,]        52
[53,]        53
[54,]        54
[55,]        55
[56,]        56
[57,]        57
[58,]        58
[59,]        59
[60,]        60
[61,]        61
[62,]   

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001  15.607813  15.626652  15.570970  15.641011  15.617142
ENSMUSG00000000028  10.396094  10.265050  10.492169  10.350183  10.414622
ENSMUSG00000000049  19.690525  19.671133  19.926724  19.894393  19.898387
ENSMUSG00000000058  10.944936  11.071840  11.101575  10.841896  11.323345
ENSMUSG00000000085  12.491665  12.279023  12.411287  12.410110  12.100900
ENSMUSG00000000088  16.550143  16.556226  16.526341  16.251088  16.208913
ENSMUSG00000000120  11.913453  11.968411  12.012620  11.668609  11.614271
ENSMUSG00000000126  10.180428  10.499808  10.067771  10.545789  10.406624
ENSMUSG00000000127  11.864985  11.693808  11.950413  11.861412  12.085904
ENSMUSG00000000142  12.022831  12.005993  11.988703  12.652696  12.438059
ENSMUSG00000000149  15.840580  15.905314  15.875935  15.977595  15.783469
ENSMUSG00000000154  15.378835  15.459865  15.528023  15.575085  15.587304
ENSMUSG00000000168  14.747357  14.7733

ENSMUSG00000001105  14.336405  14.342478  14.019747  14.058202  14.003756
ENSMUSG00000001119  11.934509  11.471585  11.682177  11.769241  12.186029
ENSMUSG00000001123  17.963254  17.946956  18.085689  18.124843  18.148980
ENSMUSG00000001127  15.121692  15.155261  15.311521  15.355752  15.446303
ENSMUSG00000001128  13.374291  13.483784  12.937673  13.150915  13.140214
ENSMUSG00000001131   9.671702   9.668679   9.287907   9.297195  10.095370
ENSMUSG00000001143  12.961420  12.812473  13.113512  13.099820  12.891477
ENSMUSG00000001155  18.387202  18.328445  18.456348  18.305741  18.271190
ENSMUSG00000001158  12.487379  12.324013  12.677610  12.322570  12.309505
ENSMUSG00000001173  11.738525  11.768100  12.160070  11.769241  12.106551
ENSMUSG00000001175  16.955149  16.976865  17.043384  17.122868  17.158005
ENSMUSG00000001211  16.687526  16.687838  16.690079  16.705452  16.668409
ENSMUSG00000001225   9.297616   9.664836   9.287907   9.297195   9.297148
ENSMUSG00000001228  10.246349  10.4056

ENSMUSG00000002249  12.588748  12.607771  12.713013  12.774927  12.550660
ENSMUSG00000002250  14.604671  14.716405  14.294012  14.873265  14.939304
ENSMUSG00000002257  10.031436   9.926592   9.978434  10.241093  10.176312
ENSMUSG00000002279  13.829339  14.152320  14.140903  14.299525  14.210928
ENSMUSG00000002289  13.475960  13.344698  14.369951  13.280983  13.472833
ENSMUSG00000002297  10.980307  11.109935  10.833901  10.773109  10.658071
ENSMUSG00000002307  12.737995  12.750789  12.787616  12.880185  12.879033
ENSMUSG00000002308  11.629426  11.602413  11.334157  11.816497  11.551132
ENSMUSG00000002324  10.868858  10.656881  11.224303  10.353400  10.544157
ENSMUSG00000002325  14.140766  14.213755  14.153254  14.419788  14.525052
ENSMUSG00000002326  12.946732  12.968189  12.764448  12.814960  12.906471
ENSMUSG00000002346  15.871310  15.901958  15.775157  15.822122  15.851241
ENSMUSG00000002365  14.176474  14.212203  14.449422  14.418960  14.469383
ENSMUSG00000002379  14.585601  14.4155

ENSMUSG00000003379  10.181779  10.347512   9.753310  10.176866   9.935245
ENSMUSG00000003402  15.903638  15.823708  15.913757  15.887039  15.788334
ENSMUSG00000003411  10.031436  10.290802   9.935216  10.241093  10.105848
ENSMUSG00000003418   9.297616   9.668679   9.753310   9.297195   9.669061
ENSMUSG00000003421  13.446207  13.614577  13.645087  13.563548  13.476974
ENSMUSG00000003429  14.847954  14.870924  14.620177  14.355198  14.431858
ENSMUSG00000003435  15.385658  15.361892  14.895819  15.211618  15.188497
ENSMUSG00000003436   9.297616   9.297080   9.287907   9.297195   9.297148
ENSMUSG00000003437  14.003054  13.869105  13.998348  13.816339  13.598419
ENSMUSG00000003444  11.470301  11.149393  11.558154  11.383948  11.660057
ENSMUSG00000003452   9.826457   9.819909   9.287907   9.297195   9.822971
ENSMUSG00000003458  14.368506  14.333521  14.214831  14.561541  14.516644
ENSMUSG00000003464  15.631090  15.749559  15.746632  15.795379  15.746131
ENSMUSG00000003476   9.939335  10.2396

ENSMUSG00000004654   9.297616   9.297080   9.287907   9.297195   9.297148
ENSMUSG00000004655  15.193828  15.189038  14.978123  15.169887  15.167267
ENSMUSG00000004665  11.896575  11.693472  11.638243  11.827313  12.102186
ENSMUSG00000004667  13.880885  14.020175  13.743610  13.630287  13.728526
ENSMUSG00000004668   9.297616   9.668679   9.287907   9.297195   9.297148
ENSMUSG00000004677  14.194056  14.197416  13.875365  13.996093  13.938884
ENSMUSG00000004709   9.671702  10.026175   9.753310   9.669329  10.090827
ENSMUSG00000004730  13.350571  13.386996  11.982457  12.935402  12.911283
ENSMUSG00000004768  12.379821  12.153298  12.392434  12.326371  12.351764
ENSMUSG00000004771  14.791498  14.888957  14.835381  14.792213  14.730437
ENSMUSG00000004788  13.339059  13.483243  13.213830  13.231133  13.256507
ENSMUSG00000004789  16.840297  16.855693  16.768346  16.803844  16.866653
ENSMUSG00000004791   9.671702   9.668679   9.287907   9.297195   9.297148
ENSMUSG00000004815  14.378136  14.3262

...

ENSMUSG00000000605  15.132801  15.094754
ENSMUSG00000000617   9.305632   9.305693
ENSMUSG00000000627   9.305632   9.305693
ENSMUSG00000000628  10.469746  10.727736
ENSMUSG00000000673  17.636466  17.619026
ENSMUSG00000000682  12.018207  11.737604
ENSMUSG00000000693  11.913068  11.601190
ENSMUSG00000000706   9.305632   9.305693
ENSMUSG00000000708  15.196461  15.211182
ENSMUSG00000000711  15.127276  15.083095
ENSMUSG00000000730   9.305632   9.305693
ENSMUSG00000000732  10.344485  10.676593
ENSMUSG00000000738  14.071947  14.163096
ENSMUSG00000000740  12.584564  12.403917
ENSMUSG00000000751  14.328405  14.331817
ENSMUSG00000000759  13.220249  13.091702
ENSMUSG00000000766   9.305632   9.809976
ENSMUSG00000000776  11.775866  11.856452
ENSMUSG00000000782  10.854801  11.173213
ENSMUSG00000000787  16.904452  16.782333
ENSMUSG00000000791  10.930379  11.107198
ENSMUSG00000000792   9.305632   9.305693
ENSMUSG00000000794   9.716557   9.305693
ENSMUSG00000000805   9.305632   9.305693
ENSMUSG000000008

ENSMUSG00000002679  12.273631  12.322745
ENSMUSG00000002699  11.711921  11.646186
ENSMUSG00000002731  13.032412  12.920692
ENSMUSG00000002733  13.279307  13.137256
ENSMUSG00000002741  15.012095  14.985587
ENSMUSG00000002748  15.342057  15.352215
ENSMUSG00000002763  15.085765  15.119178
ENSMUSG00000002767  13.912074  13.928482
ENSMUSG00000002769  19.530663  19.555409
ENSMUSG00000002771   9.716557   9.716898
ENSMUSG00000002778  15.359670  15.411066
ENSMUSG00000002797  11.953566  11.972655
ENSMUSG00000002799   9.885503   9.885950
ENSMUSG00000002803  12.084850  12.107778
ENSMUSG00000002804  12.459442  12.651740
ENSMUSG00000002814  12.075314  11.853019
ENSMUSG00000002820  14.021029  14.060526
ENSMUSG00000002846  13.374929  13.637678
ENSMUSG00000002847  13.705777  13.826431
ENSMUSG00000002870  11.072960  11.069480
ENSMUSG00000002885  12.826343  12.854347
ENSMUSG00000002900  11.913800  11.754414
ENSMUSG00000002908  11.575966  11.613356
ENSMUSG00000002944  17.335275  17.363428
ENSMUSG000000029

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  19.414155  19.393661
ENSMUSG00000010066   9.571527   9.305693
ENSMUSG00000010095  14.768771  14.876386
ENSMUSG00000010097  14.862371  14.834275
ENSMUSG00000010110  14.742743  14.804556
ENSMUSG00000010122  15.996513  15.962686
ENSMUSG00000010142  10.008658  10.182488
ENSMUSG00000010311   9.305632   9.305693
ENSMUSG00000010376  14.746360  14.710719
ENSMUSG00000010392  14.518480  14.494375
ENSMUSG00000010406  13.782782  13.612665
ENSMUSG00000010453  14.938245  14.796466
ENSMUSG00000010461  10.196796   9.716898
ENSMUSG00000010601  15.712176  15.672345
ENSMUSG00000010607  13.103519  13.051832
ENSMUSG00000010608  15.395356  15.382899
ENSMUSG00000010651  17.525427  17.486153
ENSMUSG00000010660  10.521344  10.343875
ENSMUSG00000010663  16.812090  16.826543
ENSMUSG00000010796   9.305632   9.305693
ENSMUSG00000010797  11.802511  11.809249
ENSMUSG00000010803   9.305632   9.305693
ENSMUSG00000010830   9.930801   9.608931
ENSMUSG00000010911  13.110616  12.969005
ENSMUSG00000010914  13.956929  14.

ENSMUSG00000016018  13.730592  13.570109
ENSMUSG00000016024  15.333603  15.429737
ENSMUSG00000016087  11.696341  11.642672
ENSMUSG00000016128  13.211353  13.118776
ENSMUSG00000016181  12.647226  12.592022
ENSMUSG00000016194  18.383058  18.403505
ENSMUSG00000016206  10.717625  10.920897
ENSMUSG00000016252  15.759201  15.669949
ENSMUSG00000016253  13.923848  13.759215
ENSMUSG00000016255   9.305632   9.716898
ENSMUSG00000016256  16.572736  16.521307
ENSMUSG00000016283   9.885503   9.305693
ENSMUSG00000016308  13.745685  13.580310
ENSMUSG00000016319  17.474880  17.477099
ENSMUSG00000016327   9.305632   9.305693
ENSMUSG00000016356  10.801148  10.855884
ENSMUSG00000016409  12.221019  12.297420
ENSMUSG00000016427  14.755004  14.692028
ENSMUSG00000016477  12.580011  12.645751
ENSMUSG00000016487  13.939612  13.854717
ENSMUSG00000016493  11.469591  11.493651
ENSMUSG00000016494  10.770260  10.933769
ENSMUSG00000016496  11.006981  11.040927
ENSMUSG00000016498   9.305632   9.305693
ENSMUSG000000165

In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train3vs0_time_course.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
[1] 13 21 31 39 50 60 72 73 78

$Fold02
[1]  7 10 46 53 58 59 69 76 80

$Fold03
 [1]  4  8  9 20 26 27 51 74 81 90

$Fold04
[1]  2  5 29 33 44 48 62 85 89

$Fold05
[1] 15 16 25 34 64 71 75 79 87

$Fold06
[1]  6 11 17 23 35 36 49 61 68

$Fold07
[1]  3 12 28 55 57 63 86 88 92

$Fold08
 [1] 22 24 30 37 38 43 54 65 82 83

$Fold09
[1]  1 14 19 41 47 52 56 70 77

$Fold10
[1] 18 32 40 42 45 66 67 84 91



In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1] 0 3


In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"3vs0_time_course.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls3vs0_time_course.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls3vs0_time_course.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6950703 371.3   13075691 698.4 11928728 637.1
Vcells 15983554 122.0   25842636 197.2 25842636 197.2


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls3vs0_time_course.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train3vs0_time_course.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
}

In [25]:
%%R
# print(tissue_annotation)
# print(binary_tcdd_dose_annotations)
# print(paste("Processing tissue annotation:", tissue_annotation))
# print(length(binary_tcdd_dose_detail_vec.test.cv_test_list))
# print(sum(tcdd_dose_detail.vec.train.cv_test == tissue_annotation))
# print(length(rxns))

# binary_tcdd_dose_annotations <- c(0.00, 30.00, 0.01, 0.03, 0.10, 0.30, 1.00, 3.00, 10.00)

formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00" "3.00"
[1] "0.00"
[1] "3.00"


In [26]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
#        print(length(vst.count.mtx.train.cv_train))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
       
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
#          print(tcdd_dose_detail.vec.train.cv_test)
#          print("-------------")
#          print(tissue_annotation)
#          print(binary_tcdd_dose_detail_vec.test.cv_test_list)
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
#        print(dim(cv_train.expr_mat))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.87151799687011 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.407226048533034 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.310452529994784 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.0592772812900259 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last 

[1] "Last RXN_ID =  R-MMU-9692137 : Last ARI =  0.639529569892473 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.91477005706613 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.955 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  0.959214501510574 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  0.959214501510574 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  0.959214501510574 : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  

[1] "Last RXN_ID =  R-MMU-3928657 : Last ARI =  0.561315563632692 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9034539 : Last ARI =  0.155065745603067 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  1 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.828418458781362 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  0.955555555555555 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  1 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last

[1] "Last RXN_ID =  R-MMU-354124 : Last ARI =  1 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-429415 : Last ARI =  1 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-9714691 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-420980 : Last ARI =  0.0922077243708517 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-9013004 : Last ARI =  0.959214501510574 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818576 : Last ARI =  0.261388013320005 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029273 : Last ARI =  1 : Last EC

[1] "Last RXN_ID =  R-MMU-5607725 : Last ARI =  0.813361606361904 : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-174757 : Last ARI =  1 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-187506 : Last ARI =  0.46034334141047 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.1 % remaining..."
[1] "Last RXN_ID =  R-NUL-9625422 : Last ARI =  0.32482669331473 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79 % remaining..."
[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  0.612603816345672 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  1 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =

...

[1] "Last RXN_ID =  R-MMU-9706323 : Last ARI =  0.419073002710075 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-5607750 : Last ARI =  0.0277432501723263 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.7 % remaining..."
[1] "Last RXN_ID =  R-NUL-9616944 : Last ARI =  0.398274265238193 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9698170 : Last ARI =  0.904851330203443 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-6813720 : Last ARI =  -0.0389897558878267 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-417908 : Last ARI =  0.130998083242059 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23.3 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-186509 : Last ARI =  0.0157873595976664 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9655549 : Last ARI =  0.327275120481992 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9756138 : Last ARI =  0.908510276158461 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2161293 : Last ARI =  -0.00299196117851485 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9644315 : Last ARI =  0.875709288767975 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696131 : Last ARI =  0.16379262867721 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  18.3 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-6809778 : Last ARI =  0.168475452196382 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-375339 : Last ARI =  -0.0232264344250363 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-1483002 : Last ARI =  -0.0328430864169904 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-429961 : Last ARI =  -0.0397756086760683 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-264689 : Last ARI =  1 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8858252 : Last ARI =  0.686288053936239 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  13.3 % remaining..."
[1] "Last RXN_ID =  R-M

[1] "Last RXN_ID =  R-MMU-3222259 : Last ARI =  0.0927732089870339 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5685605 : Last ARI =  0.370709871897099 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8985270 : Last ARI =  0.00713579122692318 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-192033 : Last ARI =  0.428078333333398 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6814086 : Last ARI =  0.549056826021842 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8849068 : Last ARI =  0.357595894743369 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  8.2 % remaining..."
[1] "Last RXN_

[1] "Last RXN_ID =  R-MMU-9008052 : Last ARI =  0.187969050769955 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9690439 : Last ARI =  0.149435733730019 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-1254251 : Last ARI =  0.911111111111111 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-177784 : Last ARI =  0.0417358824465305 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8850797 : Last ARI =  0.208399158296836 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-1482861 : Last ARI =  0.0758440783904896 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  3.2 % remaining..."
[1] "Last RXN_I

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls3vs0_time_course.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls3vs0_time_course.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls3vs0_time_course.Rds", sep = ""))

In [ ]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df3vs0_time_course.Rds", sep = ""))

                          RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00   R-MMU-170666        0  0.9555555556     14
R-MMU-170666.3.00   R-MMU-170671        0  0.9555555556     14
R-MMU-170671.0.00   R-MMU-170674        0  0.9555555556     14
R-MMU-170671.3.00   R-MMU-170686        0  0.9555555556     14
R-MMU-170674.0.00   R-MMU-380073        0  1.0000000000    182
R-MMU-170674.3.00   R-MMU-392187        0  0.9085102762     22
R-MMU-170686.0.00   R-MMU-392195        0  0.9085102762     22
R-MMU-170686.3.00   R-MMU-392202        0  0.9085102762     22
R-MMU-380073.0.00   R-MMU-749454        0  1.0000000000    178
R-MMU-380073.3.00   R-MMU-749456        0  1.0000000000    178
R-MMU-392187.0.00   R-MMU-751001        0  0.9492957746     21
R-MMU-392187.3.00  R-MMU-8964229        0  0.6540277778      5
R-MMU-392195.0.00  R-MMU-8964242        0  0.6540277778      5
R-MMU-392195.3.00  R-MMU-8964329        0  0.6540277778      5
R-MMU-392202.0.00  R-MMU-8982012        0  0.9048513302

R-MMU-205112.3.00  R-MMU-5654397        0  0.4705457746     16
R-MMU-205117.0.00  R-MMU-5654399        0  0.4705457746     16
R-MMU-205117.3.00  R-MMU-5654402        0  0.8105555556     20
R-MMU-209532.0.00  R-MMU-5654404        0  0.7352074967     16
R-MMU-209532.3.00  R-MMU-5654406        0  0.8105555556     18
R-MMU-209566.0.00  R-MMU-5654407        0  0.7352074967     16
R-MMU-209566.3.00  R-MMU-5654408        0  0.5574297189     12
R-MMU-9012761.0.00 R-MMU-5654409        0  0.5574297189     12
R-MMU-9012761.3.00 R-MMU-5654413        0  1.0000000000     17
R-MMU-3247837.0.00 R-MMU-5654416        0  1.0000000000     15
R-MMU-3247837.3.00 R-MMU-5654418        0  0.8270735524     14
R-MMU-3247840.0.00 R-MMU-5654422        0  0.8270735524     14
R-MMU-3247840.3.00 R-MMU-5654423        0  0.9555555556     16
R-MMU-3247843.0.00 R-MMU-5654425        0  0.8048513302     14
R-MMU-3247843.3.00 R-MMU-5654426        0  1.0000000000     18
R-MMU-3247844.0.00 R-MMU-5654428        0  0.8048513302

R-MMU-5654397.3.00 R-MMU-9753126        0  0.3023747183      5
R-MMU-5654399.0.00  R-MMU-514604        0  0.0358768214      2
R-MMU-5654399.3.00  R-MMU-975389        0  0.1757318503      4
R-MMU-5654402.0.00 R-MMU-1169394        0  0.7444444444      5
R-MMU-5654402.3.00 R-MMU-1169398        0  0.6534870665      5
R-MMU-5654404.0.00 R-MMU-1169402        0  0.6888888889      6
R-MMU-5654404.3.00 R-MMU-1169403        0  0.2465959982      4
R-MMU-5654406.0.00 R-MMU-1169405        0  0.4518070941      5
R-MMU-5654406.3.00 R-MMU-1169406        0  1.0000000000     12
R-MMU-5654407.0.00 R-MMU-1678842        0  0.9592145015     13
R-MMU-5654407.3.00 R-MMU-1678843        0  0.5606555253      5
R-MMU-5654408.0.00 R-MMU-5653754        0  0.9147700571     21
R-MMU-5654408.3.00 R-MMU-5653756        0  0.8640658317     19
R-MMU-5654409.0.00 R-MMU-9833973        0  0.8666666667      5
R-MMU-5654409.3.00  R-MMU-426240        0  0.6004657126     10
R-MMU-5654413.0.00  R-MMU-199050        0  0.4321164535

R-MMU-5654664.3.00  R-MMU-389158        0  0.7968992248      9
R-MMU-5654667.0.00  R-MMU-389348        0  0.1178575986      8
R-MMU-5654667.3.00  R-MMU-389350        0  0.1839302224      8
R-MMU-5654669.0.00  R-MMU-389352        0  0.1470367571      7
R-MMU-5654669.3.00  R-MMU-389354        0  0.1470367571      7
R-MMU-5654672.0.00  R-MMU-389381        0  0.2562123205      6
R-MMU-5654672.3.00  R-MMU-389758        0  1.0000000000     22
R-MMU-5654673.0.00  R-MMU-389759        0  0.6675753445      7
R-MMU-5654673.3.00  R-MMU-389762        0  0.4109167854      5
R-MMU-5654677.0.00  R-MMU-434836        0  0.2613249935      3
R-MMU-5654677.3.00  R-MMU-452100        0  0.5503488372      6
R-MMU-5654679.0.00 R-MMU-8852200        0  0.9147700571     17
R-MMU-5654679.3.00 R-MMU-8855381        0  0.9550000000     18
R-MMU-5654684.0.00 R-MMU-8857925        0  0.9048513302     15
R-MMU-5654684.3.00 R-MMU-9706312        0  0.4666851709      2
R-MMU-5654690.0.00 R-MMU-8849348        0  0.6738351254

R-MMU-9753126.3.00  R-MMU-450580        0  0.9147700571     12
R-MMU-514604.0.00    R-MMU-72103        0  0.9550000000     43
R-MMU-514604.3.00  R-MMU-9770119        0  0.9550000000     69
R-MMU-975389.0.00  R-MMU-9770141        0  1.0000000000    123
R-MMU-975389.3.00  R-MMU-9770142        0  1.0000000000    132
R-MMU-1169394.0.00 R-MMU-9770145        0  1.0000000000    145
R-MMU-1169394.3.00 R-MMU-9770236        0  1.0000000000    119
R-MMU-1169398.0.00 R-MMU-6798748        0  1.0000000000    106
R-MMU-1169398.3.00 R-MMU-6800434        0  1.0000000000    134
R-MMU-1169402.0.00 R-MMU-6791218        0  0.9240963855     68
R-MMU-1169402.3.00 R-MMU-6791222        0  0.9550000000     61
R-MMU-1169403.0.00 R-MMU-1605595        0  1.0000000000      3
R-MMU-1169403.3.00 R-MMU-1605624        0  0.3899891376      6
R-MMU-1169405.0.00 R-MMU-1605717        0  0.0056878687      1
R-MMU-1169405.3.00 R-MMU-6798751        0  1.0000000000    105
R-MMU-1169406.0.00 R-MMU-9840795        0  0.2515336558

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  0.5336864557      4
R-MMU-450257.0.00   R-MMU-191366        0  0.3551097281      1
R-MMU-450257.3.00   R-NUL-936401        0  0.2406230911      1
R-MMU-450333.0.00   R-MMU-744230        0 -0.0223261022      1
R-MMU-450333.3.00   R-MMU-744231        0 -0.0223261022      1
R-MMU-450474.0.00  R-MMU-9792608        0 -0.0223261022      1
R-MMU-450474.3.00  R-MMU-5694563        0  0.1030517290      1
R-MMU-4793911.0.00 R-MMU-1482543        0  0.2627346617      1
R-MMU-4793911.3.00  R-MMU-426043        0  0.2627346617      1
R-MMU-9815501.0.00 R-MMU-2872452        0  0.2083991583      1
R-MMU-9815501.3.00 R-MMU-2408515        0  0.1905221603      1
R-MMU-449803.0.00  R-MMU-9014420        0  0.0748214827      1
R-MMU-449803.3.00  R-MMU-9714480        0  0.0748214827      1
R-MMU-449811.0.00  R-MMU-9714483        0  0.0748214827      1
R-MMU-449811.3.00  R-MMU-5693681        0  0.3748224613      2
R-MMU-449855.0.00    R-MMU-73585        0  0.4457227175      1
R-MMU-449855.3.00    R-MMU-73616 

R-MMU-2024108.3.00 R-MMU-5649681        0  0.0396717802      1
R-MMU-2076383.0.00 R-MMU-5690023        0  0.4473182426      1
R-MMU-2076383.3.00 R-MMU-5577237        0  0.0461026711      5
R-MMU-2076392.0.00 R-MMU-9613264        0  0.0461026711      5
R-MMU-2076392.3.00 R-MMU-5626316        0  0.1811443667      1
R-MMU-2076419.0.00 R-MMU-1605768        0 -0.0052752366      1
R-MMU-2076419.3.00 R-MMU-4084994        0 -0.0052752366      1
R-MMU-2076508.0.00 R-MMU-9611721        0 -0.0349876814      1
R-MMU-2076508.3.00 R-MMU-9611751        0  0.1790318379      2
R-MMU-2076611.0.00 R-MMU-6814137        0 -0.0546692919      1
R-MMU-2076611.3.00  R-MMU-947673        0  0.1577174634      2
R-MMU-2684507.0.00  R-MMU-372480        0  0.0107834341      1
R-MMU-2684507.3.00  R-MMU-209738        0  0.4721956406      1
R-MMU-2731122.0.00  R-MMU-209944        0  0.4721956406      1
R-MMU-2731122.3.00 R-MMU-8963851        0  0.4721956406      1
R-MMU-2750187.0.00 R-MMU-5696091        0  0.3690696148

R-MMU-6810643.3.00  R-MMU-390886        0  0.1718505485      1
R-MMU-1433374.0.00 R-MMU-8869206        0  0.2084198909      1
R-MMU-1433374.3.00  R-MMU-196955        0  0.0509856631      1
R-MMU-1564142.0.00  R-MMU-391211        0  0.2531198739      5
R-MMU-1564142.3.00  R-MMU-435366        0  0.0572922621      1
R-MMU-1566962.0.00 R-MMU-8938300        0  0.0168302094      1
R-MMU-1566962.3.00  R-MMU-196857        0  0.1365752229      1
R-MMU-1592436.0.00 R-MMU-5696101        0  0.0951107550      1
R-MMU-1592436.3.00 R-MMU-1474146        0  0.3640658317      2
R-MMU-1604690.0.00 R-MMU-1474158        0  0.3640658317      2
R-MMU-1604690.3.00 R-MMU-1296037        0  0.0548638741      3
R-MMU-2485148.0.00 R-MMU-6797627        0  0.1795283328      1
R-MMU-2485148.3.00 R-MMU-6786650        0  0.1672377607      2
R-MMU-3827958.0.00  R-MMU-449734        0  0.4198415493      1
R-MMU-3827958.3.00  R-MMU-391940        0  0.0812737369      2
R-MMU-8943959.0.00  R-MMU-428625        0  0.2083991583

R-MMU-1358801.3.00 R-MMU-9629360        0 -0.0190615539      1
R-MMU-1963581.0.00 R-MMU-9629363        0 -0.0190615539      1
R-MMU-1963581.3.00 R-MMU-9629370        0 -0.0190615539      1
R-MMU-1963582.0.00 R-MMU-9629470        0 -0.0190615539      1
R-MMU-1963582.3.00 R-MMU-9629485        0 -0.0190615539      1
R-MMU-1963589.0.00 R-MMU-9629492        0 -0.0190615539      1
R-MMU-1963589.3.00 R-MMU-9629509        0 -0.0190615539      1
R-MMU-1977958.0.00 R-MMU-9629510        0 -0.0190615539      1
R-MMU-1977958.3.00 R-MMU-5634802        0  0.0814901275     13
R-MMU-6785636.0.00  R-MMU-446218        0  0.2414879627      1
R-MMU-6785636.3.00 R-MMU-6809624        0 -0.0095437002      1
R-MMU-8847995.0.00 R-MMU-5661240        0  0.6385917313      1
R-MMU-8847995.3.00 R-MMU-6814374        0  0.1652311237      2
R-MMU-8848005.0.00 R-MMU-8944262        0  0.2633481777      1
R-MMU-8848005.3.00 R-MMU-5686304        0  0.2757902172      2
R-MMU-8863804.0.00  R-MMU-947499        0 -0.0395594726

In [ ]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id3vs0_time_course.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id3vs0_time_course.txt",sep=""), header=FALSE)
sex <- read.table(paste(OUT_DIR, "tcdd_gender3vs0_time_course.txt", sep=""), header=FALSE)
print(sex)

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  # 绘制点，指定点的大小
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  # 添加标签
   scale_colour_manual(values = c("0" = "red", "3" = "green")) +  # 手动指定颜色
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  # 将 Section 转换为因子
     colors = c("red", "green"),  # 定义颜色
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  # 在悬停文本中显示 Project ID
)
saveWidget(p, "plotly_chart3vs0_time_course.html", selfcontained = TRUE)